In [1]:
import yaml
import pandas as pd
import os

# Go up one level to find the config file
config_path = "../config.yaml"

with open(config_path, "r") as f:
    config = yaml.safe_load(f)

# Since the paths INSIDE the config are relative to the ROOT, 
# we need to adjust them when loading data from the notebooks folder too.
data_path = os.path.join("..", config['data']['raw_path'])
df = pd.read_csv(data_path)

print("✅ Config and Data loaded successfully!")
print(f"Dataset columns: {df.columns.tolist()}")

✅ Config and Data loaded successfully!
Dataset columns: ['timestamp', 'solar_pv_output', 'wind_power_output', 'total_renewable_energy', 'solar_irradiance', 'wind_speed', 'temperature', 'humidity', 'atmospheric_pressure', 'grid_load_demand', 'frequency', 'voltage', 'power_exchange', 'battery_state_of_charge', 'battery_charging_rate', 'battery_discharging_rate', 'hour_of_day', 'day_of_week', 'predicted_solar_pv_output', 'predicted_wind_power_output', 'total_predicted_energy']


In [2]:
# Advanced time-Series Visualization

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create an interactive plot with two y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add Solar Output
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['solar_pv_output'], name="Solar PV Output (kW)"),
    secondary_y=False,
)

# Add Wind Output
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['wind_power_output'], name="Wind Power Output (kW)"),
    secondary_y=False,
)

# Add Battery State of Charge (SOC) on secondary axis
fig.add_trace(
    go.Scatter(x=df['timestamp'], y=df['battery_state_of_charge'], 
               name="Battery SOC (%)", line=dict(dash='dash', color='gray')),
    secondary_y=True,
)

fig.update_layout(title_text="Microgrid Generation vs. Battery Storage", hovermode="x unified")
fig.update_yaxes(title_text="Power Output (kW)", secondary_y=False)
fig.update_yaxes(title_text="Battery Charge (%)", secondary_y=True)
fig.show()

In [3]:
import sys
sys.path.append("..")
from src.data_loader import MicrogridDataLoader
import yaml

# Load config
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)
    # Fix paths for notebook context
    config['data']['raw_path'] = "../" + config['data']['raw_path']

# Initialize loader
loader = MicrogridDataLoader(config)
data_scaled = loader.load_and_preprocess()
X, y = loader.create_sequences(data_scaled)

print(f"Original Data Shape: {data_scaled.shape}")
print(f"Sequence Input Shape (X): {X.shape}") # Should be (Samples, 24, Features)
print(f"Target Shape (y): {y.shape}")

Standardized columns in dataset: ['timestamp', 'solar_pv_output', 'wind_power_output', 'total_renewable_energy', 'solar_irradiance', 'wind_speed', 'temperature', 'humidity', 'atmospheric_pressure', 'grid_load_demand', 'frequency', 'voltage', 'power_exchange', 'battery_state_of_charge', 'battery_charging_rate', 'battery_discharging_rate', 'hour_of_day', 'day_of_week', 'predicted_solar_pv_output', 'predicted_wind_power_output', 'total_predicted_energy']
Original Data Shape: (3546, 6)
Sequence Input Shape (X): (3522, 24, 5)
Target Shape (y): (3522,)
